# Ajuste de Curva

#### Imports

In [ ]:
from matplotlib.pyplot import rc, figure
from numpy import zeros, linspace, cos
from numpy import errstate, float64
from numpy.typing import NDArray
from itertools import pairwise
from typing import Callable

#### Definições de Tipo

In [ ]:
NDArray = NDArray[float64]
Function = Callable[[NDArray], NDArray]

#### Ajuste de curva

In [ ]:
def adjust(D: NDArray, I: NDArray, degree: int):
  with errstate(all = "raise"):
    G = D[:, None] ** range(degree)
    L = zeros((degree, degree))
    U, A = (G.T @ G), (G.T @ I)
    L.flat[::(degree + 1)] = 1
    for i, ii in pairwise(range(degree)):
      L[ii:, i] = U[ii:, i] / U[i, i]
      U[ii:] -= L[ii:, i, None] * U[i]
    x, y = zeros(degree), zeros(degree)
    for i in range(degree):
      y[i] = A[i] - (L[i] @ y)
    for i in reversed(range(degree)):
      temp = y[i] - (U[i] @ x)
      x[i] = temp / U[i, i]
    return (G @ x)

#### Plotagem de gráficos

In [ ]:
def plt_img(D: NDArray, I: NDArray, R: NDArray):
  rc("font", family = "Arial", size = 10)
  rc("lines", aa = True, lw = .5)
  fig = figure(None, (12, 5), 200)
  axe = fig.subplots()

  axe.set_title("Ajuste de Curva")
  axe.plot(D, I, label = "Original")
  axe.plot(D, R, label = "Ajustado")
  axe.set_xlabel("Domínio")
  axe.set_ylabel("Imagem")
  axe.legend()

  fig.tight_layout(pad = 1)

#### Função main

In [ ]:
def main(f: Function, a: float, b: float, p: int, n: int):
  plt_img((D := linspace(a, b, p)), (I := f(D)), adjust(D, I, n))

#### Execução do Código

In [ ]:
main(lambda x: cos(x) + cos(2.5 * x) + cos((2**.35) * x), -15, 15, 2000, 65)